In [1]:
# Importing the libraries

import numpy as np
print('Numpy version: ', np.__version__)

import matplotlib.pyplot as plt

import mne
print('MNE-Python: ', mne.__version__)

mne.set_log_level('error')  # reduce extraneous MNE output

Numpy version:  1.24.3
MNE-Python:  1.4.2


## Import the filtered dataset

In [2]:
filters = list()
file_path = "D:\IIIT_Hyderabad\Code\data_filter\sub-"
suffix = "_ses-t1_task-resteyesc_eeg_filter.fif"
for i in range(111):
    j = str(i+1)
    file_name = file_path+j+suffix
    filtered = mne.io.read_raw_fif(file_name)
    # Set montage
    montage = 'standard_1005'  # Electrode position file
    filtered.set_montage(montage)
    filters.append(filtered)

In [3]:
len(filters)

111

## Applying ICA

In [4]:
from mne_icalabel import label_components
from autoreject import get_rejection_threshold

In [5]:
# Afunction that:
# 1. Interpolates bad channels
# 2. Applies high-pass filter of 1 Hz
# 3. Segments the data into 2s epochs

def ica_data(data):
    data = data.load_data().copy().interpolate_bads(reset_bads=False) # Interpolating bad channels
    raw_ica = data.load_data().copy().filter(l_freq = 1, h_freq = None) # Applying high-pass filter
    # Segment the data
    tstep = 2.0
    events_ica = mne.make_fixed_length_events(raw_ica, duration=tstep)
    epochs_ica = mne.Epochs(raw_ica, events_ica, tmin=0.0, tmax=tstep, baseline=None, preload=True)

    return epochs_ica

#epochs = list()

In [6]:
# A function that applies
# 1. Autoreject to get the amplitude threshold
# 2. ica with components 20, and method infomax
# 3. ica_label and print the label for each component 

def ica_(epochs_ica):
    reject = get_rejection_threshold(epochs_ica)
    ica = mne.preprocessing.ICA(n_components = 20, 
                                max_iter = 200, 
                                method='infomax', 
                                fit_params=dict(extended=True), 
                                random_state = 42)
    ica.fit(epochs_ica, reject = reject, tstep = 2.0)
    return ica



In [7]:
def ica_labels(epochs_ica,ica):
    ic_labels = label_components(epochs_ica, ica, method="iclabel")
    return ic_labels['labels']

labels = list()

In [8]:
def clean_ica(data,ica, label):   
    exclude_idx = [idx for idx, source in enumerate(label) if source not in ["brain"]]
    reconst_data = data.copy()
    reconst_data = ica.apply(reconst_data, exclude=exclude_idx)
    return reconst_data

datas = list()

In [9]:
icas = list()

In [10]:
prefix = "D:\IIIT_Hyderabad\Code\ica_file\sub-"
suffix = "_ses-t1_task-resteyesc_eeg_ica.fif"
for i in range(94):
    j = str(i+1)
    filename = prefix+j+suffix
    ica = mne.preprocessing.read_ica(filename)
    icas.append(ica)

In [11]:
len(icas)

94

In [12]:
for i in range(94,111):
    data = filters[i]
    epochs_ica = ica_data(data)
    print('Participant: ',i+1)
    ica = ica_(epochs_ica)
    label = ica_labels(epochs_ica, ica)
    new_data = clean_ica(data,ica,label)
    icas.append(ica)
    labels.append(label)
    datas.append(new_data)
    j = str(i+1)
    ica.save('D:\IIIT_Hyderabad\Code\ica_file\sub-'+j+'_ses-t1_task-resteyesc_eeg_ica.fif', overwrite = True)
    new_data.save("D:\IIIT_Hyderabad\Code\preprocessed_data\sub-"+j+'_ses-t1_task-resteyesc_eeg_ica_filtered'+'.fif', overwrite=True)

Participant:  95
Estimating rejection dictionary for eeg
Participant:  96
Estimating rejection dictionary for eeg
Participant:  97
Estimating rejection dictionary for eeg
Participant:  98
Estimating rejection dictionary for eeg
Participant:  99
Estimating rejection dictionary for eeg
Participant:  100
Estimating rejection dictionary for eeg
Participant:  101
Estimating rejection dictionary for eeg
Participant:  102
Estimating rejection dictionary for eeg
Participant:  103
Estimating rejection dictionary for eeg
Participant:  104
Estimating rejection dictionary for eeg
Participant:  105
Estimating rejection dictionary for eeg
Participant:  106
Estimating rejection dictionary for eeg
Participant:  107
Estimating rejection dictionary for eeg
Participant:  108
Estimating rejection dictionary for eeg
Participant:  109
Estimating rejection dictionary for eeg
Participant:  110
Estimating rejection dictionary for eeg
Participant:  111
Estimating rejection dictionary for eeg


In [12]:
raw = mne.io.read_raw_edf("D:\IIIT_Hyderabad\Code\data\sub-6_ses-t1_task-resteyesc_eeg.edf")
raw.plot()
filtered = mne.io.read_raw_fif("D:\IIIT_Hyderabad\Code\preprocessed_data\sub-6_ses-t1_task-resteyesc_eeg_ica_filtered.fif")
filtered.plot()